# Train the neural network
In this paper, the [YOLOv3](https://arxiv.org/abs/1804.02767) neural network is used for fish detection. The implementation from [Ultralytics](https://github.com/ultralytics/yolov3) in Pytorch was for us the most conventient to use.

## Setup GPU
When using Google Colab, setup the hardware accelerator to use a GPU by:

**Edit** > **Notebook settings** > **Hardware accelerator**.

![Change Colab to use GPU](colab_gpu.png)

## Install software
To install the software on your own computer, follow the steps provided in the [readme](https://github.com/Rick-v-E/automatic_discard_registration/blob/master/README.md). If running on Google Colab, clone the GIT repository and install it's dependencies:

In [ ]:
%%shell

# Check if the repository is already available, if not, clone and install
if [ ! -d .git ]
then
  git clone https://github.com/Rick-v-E/automatic_discard_registration.git
  pip install -r automatic_discard_registration/requirements.txt
  pip install -r automatic_discard_registration/detection/yolov3/requirements.txt
  pip install automatic_discard_registration/detection/apex
  pip install gdown
else
  git pull
fi

If you installed the software in the previous step, enter the repository:

In [ ]:
%cd automatic_discard_registration

## Setup dataset
The complete dataset can be downloaded from [4TU.ResearchData](https://doi.org/10.4121/16622566.v1). To use this dataset, extract both `fdf_images.zip` and `results.zip` in the [data](https://github.com/Rick-v-E/automatic_discard_registration/tree/master/data) folder.

For use on Google Colab, we have created a smaller subset of the data. This dataset contains only part of the images of the complete dataset, but contains all result from the complete dataset.

---
**IMPORTANT**

Execute only one of the three cells below! Each cell contains a method to import the data, if one method fails, use another method. If the method succeed, go to the next section in this notebook.

---

**METHOD 1** Download and extract the sample dataset (this will take around 5-10 minutes):

In [ ]:
!gdown --id 1TcyeeX0UjhWldbjhLkCRJIuktDNeAMJJ
!unzip -q fdf_sample_dataset.zip -d data
!rm fdf_sample_dataset.zip

**METHOD 2** Download the [sample dataset](https://drive.google.com/file/d/1TcyeeX0UjhWldbjhLkCRJIuktDNeAMJJ/view?usp=sharing) manually and upload it to Google Colab in the `automatic_discard_registration` opening the files tab and right click on the folder name:

![Manual upload image](colab_manual_upload.png)

After uploading, extract the dataset:

In [ ]:
!unzip -q fdf_sample_dataset.zip -d data
!rm fdf_sample_dataset.zip

**METHOD 3** Download the [sample dataset](https://drive.google.com/file/d/1TcyeeX0UjhWldbjhLkCRJIuktDNeAMJJ/view?usp=sharing) and upload it to your personal Google Drive account. Connect this account to Google Colab:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -q ../drive/MyDrive/fdf_sample_dataset.zip -d data

Check if the dataset is loaded correctly:

In [ ]:
from pathlib import Path

DATA_PATH = Path("data")
NEEDED_FOLDERS = ["fdf_images", "results"]

# Check if all folders are correct
if not all([(DATA_PATH / f).is_dir() for f in NEEDED_FOLDERS]):
    print("Could not find all data folders! Did you extract both fdf_images.zip and results.zip in the data folder?")

To get the same results as in the paper, use the complete dataset. Upload the dataset to your Google Drive and [mount](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166) this folder to your Google Colab environment. 

## Setup training notebook
Start by importing the dependencies:

In [ ]:
%load_ext tensorboard
%matplotlib inline

import math
import torch
import warnings

from pathlib import Path
from matplotlib import pyplot as plt

from detection import setup_paths
from detection.train import train
from detection.data_loader import FDFLoader

from common.nb_utils import show_random_image
from detection.yolov3.models import attempt_download
from detection.yolov3.utils.utils import load_classes

warnings.filterwarnings("ignore", category=UserWarning)

Check if we have a GPU available:

In [ ]:
if not torch.cuda.is_available():
    print("No GPU device found! If you are working on Google Colab, make sure that you select the GPU hardware accelerator in the notebook settings.")

## Setup all model parameters
Now, define all the model parameters. There are hyperparameters for the model and configuration parameters:

In [ ]:
hyp = {
    "giou": 3.54,  # giou loss gain
    "cls": 37.4,  # cls loss gain
    "cls_pw": 1.0,  # cls BCELoss positive_weight
    "obj": 64.3,  # obj loss gain (*=img_size/320 if img_size != 320)
    "obj_pw": 1.0,  # obj BCELoss positive_weight
    "iou_t": 0.20,  # iou training threshold
    "lr0": 0.01,  # initial learning rate (SGD=5E-3, Adam=5E-4)
    "lrf": 0.0005,  # final learning rate (with cos scheduler)
    "momentum": 0.937,  # SGD momentum
    "weight_decay": 0.0005,  # optimizer weight decay
    "fl_gamma": 0.0,  # focal loss gamma (efficientDet default is gamma=1.5)
    "hsv_h": 0.0138,  # image HSV-Hue augmentation (fraction)
    "hsv_s": 0.678,  # image HSV-Saturation augmentation (fraction)
    "hsv_v": 0.36,  # image HSV-Value augmentation (fraction)
    "degrees": 1.98 * 0,  # image rotation (+/- deg)
    "translate": 0.05 * 0,  # image translation (+/- fraction)
    "scale": 0.05 * 0,  # image scale (+/- gain)
    "shear": 0.641 * 0,
} 

opt = {
    "epochs": 100,  # Number of epochs. We did 800 epochs on complete dataset
    "batch_size": 8, 
    "multi_scale": False,  # adjust (67%% - 150%%) img_size every 10 batches
    "img_size": [320, 640, 416],  # [min-train, max-train, test] image size
    "rect": False,  # rectangular training
    "resume": False,  # resume traning from last.pt
    "nosave": False,  # only save final epoch
    "notest": False,  # only test final epoch
    "evolve": False,  # evolve hyperparameters
    "bucket": "",  # gsutil bucket
    "cache_images": False,  # cache images for faster training
    "name": "",  # renames results.txt to results_name.txt if supplied
    "adam": False,  # use adam optimizer
    "single_cls": False,  # train as single-class dataset
    "freeze_layers": False,  # freeze non-output layers,
    "name": "FDF_training",    
    "device": "",
}

## Load the dataset
Now, load the dataset from the data folder. First, load the folders as variables:

In [ ]:
data_folder = Path("data")
weights_folder = data_folder / "results/model_weights"
dataset_folder = data_folder / "fdf_images"

# Define files
paths = {
    "weights_folder": data_folder,
    "start_weights_file": weights_folder / "yolov3-spp-ultralytics.pt",
    "best_weights_file": data_folder / "best.pt",
    "last_weights_file": data_folder / "last.pt",
    "results_file": data_folder / "training_results.txt",
    "cfg_file": Path("detection") / "yolov3-spp-fdf.cfg",
    "names_file": Path("detection") / "fish_classes.names",
}

# Create folder if not exists
weights_folder.mkdir(parents=True, exist_ok=True)

# Check if files exists
assert paths["cfg_file"].is_file()
assert paths["names_file"].is_file()
assert dataset_folder.is_dir()

Try to download the pre-trained weights file from [Ultralytics](https://drive.google.com/drive/folders/1LezFG5g3BCW6iYaV89B2i64cqEUZD7e0). If this does not work, try to install `curl`:

`sudo apt-get install curl`

In [ ]:
attempt_download(str(paths["start_weights_file"]))

Load the image classes:

In [ ]:
classes = load_classes(paths["names_file"])
print(classes)

Now, we can load the dataset images and annotations. Since we are using a custom annotation format (containing addional information like fish orientation, occlusion and an unique id), we use our custom dataloader:

In [ ]:
# Check and recalculate image size
imgsz_min, imgsz_max, imgsz_test = opt["img_size"] 

# Image Sizes
gs = 32  # (pixels) grid size
assert math.fmod(imgsz_min, gs) == 0, "--img-size %g must be a %g-multiple" % (imgsz_min, gs)
opt["multi_scale"] |= imgsz_min != imgsz_max  # multi if different (min, max)
if opt["multi_scale"]:
    if imgsz_min == imgsz_max:
        imgsz_min //= 1.5
        imgsz_max //= 0.667
    grid_min, grid_max = imgsz_min // gs, imgsz_max // gs
    imgsz_min, imgsz_max = int(grid_min * gs), int(grid_max * gs)
img_size = imgsz_max 

train_dataset = FDFLoader(
    dataset_folder,
    classes,
    subset_name="train",
    img_size=img_size,
    batch_size=opt["batch_size"],
    augment=True,
    hyp=hyp,
    rect=opt["rect"],
    cache_images=opt["cache_images"],
    single_cls=opt["single_cls"],
)
test_dataset = FDFLoader(
    dataset_folder,
    classes,
    subset_name="validation",
    img_size=imgsz_test,
    batch_size=min(opt["batch_size"], len(train_dataset)),
    hyp=hyp,
    rect=True,
    cache_images=opt["cache_images"],
    single_cls=opt["single_cls"],
)

Now, show an random training and testing image with annotations to make sure that we have loaded the dataset correctly:

In [ ]:
f, axarr = plt.subplots(1, 2, figsize=(20,20))
show_random_image(train_dataset, axarr[0], classes)
show_random_image(test_dataset, axarr[1], classes)
plt.show()

## Train the model
First step is to launch Tensorboard:

In [ ]:
print('Start Tensorboard with "tensorboard --logdir=data/runs --bind_all", view at http://localhost:6006/')
tb = torch.utils.tensorboard.SummaryWriter(comment=opt["name"], log_dir="data/runs/{}".format(opt["name"]))
%tensorboard --logdir data/runs/FDF_training

Start the training:

In [ ]:
train(train_dataset, test_dataset, paths, hyp, opt, tb_writer=tb)

The best epoch was selected using the accuracy. The corresponding weights are saved in `data/best.pt`. For the next notebooks, we are using the weights trained on the complete dataset.